# Google Colab
Use this cell only if you runnning the notebook in google colab

---



In [1]:
import urllib.request

print('Beginning file download at super fast speeds...')

url = "https://storage.googleapis.com/kaggle-datasets/23079/29550/asl_alphabet_train.zip?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1567093454&Signature=ZgE2LifsncdLIC7SBxw%2BevpUt7lh7S0w8TfkD2PD7ZaokLWrk9wgdw4wW4l10gXQbS7IQnmUxzXOL%2FI0w38PvcwQhLbtRx%2FwM%2F97otvGy7k09tyAmXbHnhw%2F3kj0ReS2avF3DK7HK8P2mC0Hku8qKprYnHKrxIqONBzseJ%2FKIS8xjtgPqi%2B3hDM%2BYrhJebiu9FV8oBB%2Fi5n%2F8ssD9RqubTTA8oP37AhUOgpOu9cUJOtp%2BYZJa09ZyIzAlVts1miL%2B0kGf9rfp4JLqKkI2WB9BS3d8F9z%2BcJADCnqhT7uP2K%2F67WL%2FVU1kH8%2ByUv%2BNWZ%2Fs5%2BQkmSSD08qE3UcxFVbLA%3D%3D"
urllib.request.urlretrieve(url, './train.zip')

!unzip 'train.zip'

Beginning file download at super fast speeds...


('./train.zip', <http.client.HTTPMessage at 0x7fc5eedc8048>)

# Importing all the pre-requisite libraries

---



In [3]:
from keras.applications.resnet50 import ResNet50
from sklearn import metrics
import copy
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import keras
import cv2
import shutil
import tensorflow as tf
from keras.layers import LeakyReLU
from keras.metrics import categorical_accuracy
from keras.utils import plot_model, to_categorical
from keras.models import Model,load_model, Sequential
from keras.models import load_model
from keras.layers import Input, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D
from keras.layers import AveragePooling2D, MaxPooling2D, Dropout, GlobalMaxPooling2D, GlobalAveragePooling2D
from keras import regularizers
from keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.


# Neural Network Modelling
Using **Resnet50** as the base model for the neural network framework.
Then adding 3 fully connected layers of 1024 neurons with *relu* activation function and adding the last output layer using *softmax* activation with 29 nuerons.


---



In [4]:
img_height,img_width=(200,200)
base_model = ResNet50(weights= 'imagenet', include_top=False, input_shape= (200,200,3))
for layer in base_model.layers:
    layer.trainable = False
num_classes=29
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)
x = Dense(1024, activation='relu')(x)
# x = Dropout(0.3)(x)
x = Dense(1024, activation='relu')(x)
# x = Dropout(0.3)(x)
x = Dense(1024, activation='relu')(x)
# x = Dropout(0.3)(x)
predictions = Dense(num_classes, activation= 'softmax')(x)
model = Model(inputs = base_model.input, outputs = predictions)

W0826 15:52:07.393628 140488690247552 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0826 15:52:07.432772 140488690247552 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0826 15:52:07.441434 140488690247552 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4185: The name tf.truncated_normal is deprecated. Please use tf.random.truncated_normal instead.

W0826 15:52:07.477871 140488690247552 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W0826 15:52:07.479602

94658560/94653016 [==============================] - 3s 0us/step


W0826 15:52:21.383858 140488690247552 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3445: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


# Learning Algorithm
Using adam optimizer with loss as categorical crossentropy and printing out accuracy as metrics while learning.

*Define the directory if you are running notebook on local machine*


---


## Train 1
First the model is trained on the Resnet for 10 epochs and batch size of 32

---



In [0]:
model.compile(optimizer='adam', loss='categorical_crossentropy',metrics=['accuracy'])

EPOCHS = 10
BS = 32
aug = ImageDataGenerator(rotation_range=5, zoom_range=0.2, rescale=1./255,
                         width_shift_range=0.2, height_shift_range=0.2,
                         shear_range=0.18,
                         horizontal_flip=False, fill_mode="nearest")

#directory for image as data
directory = "/content/asl_alphabet_train"

image_generator = aug.flow_from_directory(directory, target_size=(200, 200), 
                    class_mode='categorical',batch_size=32)

H = model.fit_generator(image_generator,steps_per_epoch=2718, epochs=EPOCHS).cuda()

In [0]:
for layer in model.layers:
    layer.trainable = True

## Train 2
Then it is trained on the 3 additional fully connected layers for 2 epochs and batch size of 32

---



In [0]:
model.compile(optimizer='adam', loss='categorical_crossentropy',metrics=['accuracy'])

EPOCHS = 2
BS = 32
aug = ImageDataGenerator(rotation_range=5, zoom_range=0.2, rescale=1./255,
                         width_shift_range=0.2, height_shift_range=0.2,
                         shear_range=0.18,
                         horizontal_flip=False, fill_mode="nearest")

#directory for image as data
directory = "/content/asl_alphabet_train"

image_generator = aug.flow_from_directory(directory, target_size=(200, 200), 
                    class_mode='categorical',batch_size=32)

H2 = model.fit_generator(image_generator,steps_per_epoch=2718, epochs=EPOCHS)

## Final steps
Saving the model and printing history of first and second train


---



In [0]:
model.save("./resnet1.h5")

H.history

H2.history